In [8]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.3 MB/s eta 0:00:00


In [9]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [11]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import faiss
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import os

In [2]:
df= pd.read_csv('/content/hotels_reviews_with_sentiment.csv')

In [3]:
df.head()

,City,Hotel Name,Address,Rating,Price Level,Total Reviews,Phone Number,Reviewer,Review Text,Review Rating,Google Maps Link,Cleaned Review Text,Predicted Sentiment,embeddings
0,Lahore,"Pearl Continental Hotel, Lahore","Shahrah-e-Quaid-e-Azam, Garhi Shahu, Lahore, P...",4.4,Not Provided,41021,+92 42 111 505 505,Abdul Basit Latif,I had an amazing experience at this hotel\nThe...,5,https://maps.google.com/?cid=15495242241067655383,amazing experience hotel management exceptiona...,POSITIVE,[-3.07017211e-02 5.13005629e-02 5.08749066e-...
1,Lahore,"Pearl Continental Hotel, Lahore","Shahrah-e-Quaid-e-Azam, Garhi Shahu, Lahore, P...",4.4,Not Provided,41021,+92 42 111 505 505,Misal Fatima,I visited the PC hotel and the rooms were beau...,5,https://maps.google.com/?cid=15495242241067655383,visited pc hotel room beautifully appointed cl...,POSITIVE,[ 3.33876684e-02 -9.79979639e-04 -1.29095353e-...
2,Lahore,"Pearl Continental Hotel, Lahore","Shahrah-e-Quaid-e-Azam, Garhi Shahu, Lahore, P...",4.4,Not Provided,41021,+92 42 111 505 505,Suhail Ahmed,I recently had the pleasure of staying at the ...,5,https://maps.google.com/?cid=15495242241067655383,recently pleasure staying pearl continental ho...,POSITIVE,[ 2.53267568e-02 -3.63543909e-03 7.81602934e-...
3,Lahore,"Pearl Continental Hotel, Lahore","Shahrah-e-Quaid-e-Azam, Garhi Shahu, Lahore, P...",4.4,Not Provided,41021,+92 42 111 505 505,fiza tirmzi,Had a lovely stay at PC Lahore. The rooms do n...,4,https://maps.google.com/?cid=15495242241067655383,lovely stay pc lahore room need upgrade rememb...,POSITIVE,[ 5.67785290e-04 2.83062831e-02 9.28273126e-...
4,Lahore,"Pearl Continental Hotel, Lahore","Shahrah-e-Quaid-e-Azam, Garhi Shahu, Lahore, P...",4.4,Not Provided,41021,+92 42 111 505 505,Awais Mughal,I visited PC hotel. I had a pleasant experienc...,5,https://maps.google.com/?cid=15495242241067655383,visited pc hotel pleasant experience hotel ser...,POSITIVE,[ 3.88436839e-02 3.01688034e-02 6.03495240e-...


In [4]:
data=df[df["City"]=="Islamabad"]

In [5]:
data.head()

,City,Hotel Name,Address,Rating,Price Level,Total Reviews,Phone Number,Reviewer,Review Text,Review Rating,Google Maps Link,Cleaned Review Text,Predicted Sentiment,embeddings
1064,Islamabad,Islamabad Marriott Hotel,"Agha Khan Rd, F-5/1 F-5, Islamabad, 44000, Pak...",4.6,Not Provided,25005,+92 51 2826121,Hazem Malek,What an exceptional experience we had at the M...,5,https://maps.google.com/?cid=9171730565047068216,exceptional experience marriott islamabad mome...,POSITIVE,[-8.12390354e-03 3.50867286e-02 1.20872455e-...
1065,Islamabad,Islamabad Marriott Hotel,"Agha Khan Rd, F-5/1 F-5, Islamabad, 44000, Pak...",4.6,Not Provided,25005,+92 51 2826121,Ibrahim Syed,"When we discuss service experience, cleanlines...",3,https://maps.google.com/?cid=9171730565047068216,discus service experience cleanliness key fact...,POSITIVE,[-3.22491825e-02 -9.51885711e-03 5.98738901e-...
1066,Islamabad,Islamabad Marriott Hotel,"Agha Khan Rd, F-5/1 F-5, Islamabad, 44000, Pak...",4.6,Not Provided,25005,+92 51 2826121,Md Masud,I recently stayed at the *Islamabad Marriott H...,5,https://maps.google.com/?cid=9171730565047068216,recently stayed islamabad marriott hotel thoro...,POSITIVE,[-5.52569225e-04 2.65779942e-02 1.35404719e-...
1067,Islamabad,Islamabad Marriott Hotel,"Agha Khan Rd, F-5/1 F-5, Islamabad, 44000, Pak...",4.6,Not Provided,25005,+92 51 2826121,Mohammed Z,The positive:\nThis hotel is a best choice if ...,4,https://maps.google.com/?cid=9171730565047068216,positive hotel best choice traveling business ...,POSITIVE,[ 7.20246211e-02 2.81622019e-02 1.70947686e-...
1068,Islamabad,Islamabad Marriott Hotel,"Agha Khan Rd, F-5/1 F-5, Islamabad, 44000, Pak...",4.6,Not Provided,25005,+92 51 2826121,Christian Sanjaya,One of the best hotel I’ve ever stayed in. The...,5,https://maps.google.com/?cid=9171730565047068216,one best hotel ive ever stayed hospitality bes...,POSITIVE,[ 4.82369550e-02 3.25636268e-02 -9.24826879e-...


In [6]:
data['City'].unique()

array(['Islamabad'], dtype=object)

In [7]:
data.to_csv("Islamabad.csv",index=False)

In [12]:
df = pd.read_csv("/content/Islamabad_ground_truth.csv",on_bad_lines='skip')
df['Cleaned Review Text'] = df['Cleaned Review Text'].fillna("").astype(str)
tqdm.pandas()

In [26]:
def parse_embedding(text):
    try:
        return np.fromstring(text.strip("[]"), sep=" ")
    except:
        return np.array([])


In [27]:
df['embeddings']=df['embeddings'].apply(parse_embedding)

In [17]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
def calc_precision(results, rel_retrieved):
    if len(results) == 0:
        return 0
    precision= rel_retrieved/len(results)
    return precision

def calc_recall(rel_retrieved, total_rel):
    if total_rel == 0:
        return 0
    recall = rel_retrieved/total_rel
    return recall

def calc_f1_score(precision, recall):
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

In [93]:
def search_reviews(query, sentiment_filter="POSITIVE", top_k=40):
    """
    Perform semantic hotel review search using FAISS (L2 distance) and sentiment filtering.

    Parameters:
        query (str): User's search query.
        sentiment_filter (str): Desired sentiment ("POSITIVE" or "NEGATIVE").
        top_k (int): Number of top results to return.

    Returns:
        None (prints evaluation metrics and statistics)
    """
    review_vectors = np.vstack(df['embeddings'].values).astype('float32')
    dim = review_vectors.shape[1]

    index = faiss.IndexFlatL2(dim)
    index.add(review_vectors)

    query_vector = embedder.encode(query).astype('float32').reshape(1, -1)

    distances, indices = index.search(query_vector, len(review_vectors))

    results = []
    result_ids = set()

    for idx in indices[0]:
        row = df.iloc[idx]

        if str(row.get('Predicted Sentiment', '')).upper() == sentiment_filter.upper():
            if row['id'] not in result_ids:
                results.append(row)
                result_ids.add(row['id'])

        if len(results) >= top_k:
            break


    rel_retrieved = 0
    for row in results:
        if any(query.lower() in str(row.get(f'label_{i}', '')).lower() for i in range(1, 6)):
            rel_retrieved += 1


    relevant_ids = set()
    for i in range(1, 6):
        if f'label_{i}' in df.columns:
            matched_rows = df[df[f'label_{i}'].astype(str).str.lower() == query.lower()]
            relevant_ids.update(matched_rows['id'].tolist())
    total_rel = len(relevant_ids)

    
    precision=calc_precision(results,rel_retrieved)
    recall=calc_recall(rel_retrieved,total_rel)
    f1_score=calc_f1_score(precision,recall)

    if results:
        print(f"\nPrecision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1_score:.4f}")
        print(f"Relevant Retrieved: {rel_retrieved}")
        print(f"Total Relevant in Dataset: {total_rel}")
        print(f"Total Retrieved: {len(results)}")
    else:
        print("\nNo results found matching the sentiment filter.")


In [94]:
queries=df["label_1"].unique()

In [95]:
for i in range(0,10):
  print(f"\n\n{i+1} Query: {queries[i]}\n")
  search_reviews(str(queries[i]))



1 Query: friendly staff


Precision: 0.2250
Recall: 0.0657
F1 Score: 0.1017
Relevant Retrieved: 9
Total Relevant in Dataset: 137
Total Retrieved: 40


2 Query: excellent service


Precision: 0.1000
Recall: 0.1429
F1 Score: 0.1176
Relevant Retrieved: 4
Total Relevant in Dataset: 28
Total Retrieved: 40


3 Query: comfortable beds


Precision: 0.1000
Recall: 0.2222
F1 Score: 0.1379
Relevant Retrieved: 4
Total Relevant in Dataset: 18
Total Retrieved: 40


4 Query: clean rooms


Precision: 0.5000
Recall: 0.1408
F1 Score: 0.2198
Relevant Retrieved: 20
Total Relevant in Dataset: 142
Total Retrieved: 40


5 Query: nan


Precision: 1.0000
Recall: 0.0344
F1 Score: 0.0664
Relevant Retrieved: 40
Total Relevant in Dataset: 1164
Total Retrieved: 40


6 Query: poor management


Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Relevant Retrieved: 0
Total Relevant in Dataset: 15
Total Retrieved: 40


7 Query: great location


Precision: 0.1250
Recall: 0.1163
F1 Score: 0.1205
Relevant Retrieved: 5
To